<a href="https://colab.research.google.com/github/Oc941nizet/Assignement_Oc-ane-Nizet-/blob/main/Assignment_Oc%C3%A9ane_Nizet_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1.Reading dataset


Training data

In [1]:
import pandas as pd

train_df = pd.read_csv('https://raw.githubusercontent.com/aws-samples/aws-machine-learning-university-accelerated-nlp/master/data/final_project/imdb_train.csv', header=0)
train_df.head()

,text,label
0,This movie makes me want to throw up every tim...,0
1,Listening to the director's commentary confirm...,0
2,One of the best Tarzan films is also one of it...,1
3,Valentine is now one of my favorite slasher fi...,1
4,No mention if Ann Rivers Siddons adapted the m...,0


Test data

In [2]:
import pandas as pd

test_df = pd.read_csv('https://raw.githubusercontent.com/aws-samples/aws-machine-learning-university-accelerated-nlp/master/data/final_project/imdb_test.csv', header=0)
test_df.head()

,text,label
0,What I hoped for (or even expected) was the we...,0
1,Garden State must rate amongst the most contri...,0
2,There is a lot wrong with this film. I will no...,1
3,"To qualify my use of ""realistic"" in the summar...",1
4,Dirty War is absolutely one of the best politi...,1


In [3]:
train_df["label"].value_counts()

,count
label,
0,12500
1,12500


In [4]:
# Install the library and functions
import nltk

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [5]:
import nltk, re
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize

# Let's get a list of stop words from the NLTK library
stop = stopwords.words('english')

# These words are important for our problem. We don't want to remove them.
excluding = ['bad, worse', 'amazing','great', 'again', 'fun', 'well', 'excellent',
             'brilliant', 'outstanding', 'fantastic', 'good', 'wonderful', 'boring',
             'terrible', 'awful', 'disappointing', 'horrible', 'mediocre', 'average',
             'decent', 'fair', 'okay', 'impressive', 'superb', 'delightful',
             'enjoyable', 'stunning', 'incredible', 'marvelous', 'enthralling',
             'engaging', 'terrific', 'heartwarming', 'dreadful', 'pathetic', 'lame', 'predictable', 'forgettable', 'weak',
             'unbearable', 'lousy', 'tedious','cheesy', 'uninpiring', 'cringeworthy', 'Mediocre', 'Average', 'decent',
             'fair', 'okay', 'passable', 'plain', 'predictable', 'unremarkable', 'ordinary', 'acceptable', 'fine', 'mild',
             'standard', 'moderate', 'captivating', 'mesmerizing', 'uplifting', 'thrilling', 'breathtaking', 'magnificent',
             'extraordinary', 'refreshing', 'riveling', 'hilarous', 'charming', "Top-notch", "Triumphant", "Masterful", "Inspiring",
    "Remarkable", "Splendid", "Memorable",  "Miserable", "Painful", "Atrocious", "Disgusting", "Banal", "Soulless",
    "Overrated", "Mediocre", "Horrid", "Distasteful", "Disastrous", "Shallow",
    "Clichéd", "Confusing", "Frustrating", "Lifeless", "Usual", "Typical", "Adequate",
    "Sufficient", "Regular", "Competent", "Unassuming", "Balanced",
    "Inoffensive", "Neutral", "Middle-of-the-road", "Workmanlike",
    "Routine", "Unspectacular", "Modest"]

# New stop word list
stop_words = [word for word in stop if word not in excluding]

snow = SnowballStemmer('english')

def process_text(texts):
    final_text_list=[]
    for sent in texts:

        # Check if the sentence is a missing value
        if isinstance(sent, str) == False:
            sent = ""

        filtered_sentence=[]

        sent = sent.lower() # Lowercase
        sent = sent.strip() # Remove leading/trailing whitespace
        sent = re.sub('\s+', ' ', sent) # Remove extra space and tabs
        sent = re.compile('<.*?>').sub('', sent) # Remove HTML tags/markups:

        for w in word_tokenize(sent):
            # We are applying some custom filtering here, feel free to try different things
            # Check if it is not numeric and its length>2 and not in stop words
            if(not w.isnumeric()) and (len(w)>2) and (w not in stop_words):
                # Stem and add to filtered list
                filtered_sentence.append(snow.stem(w))
        final_string = " ".join(filtered_sentence) #final string of cleaned words

        final_text_list.append(final_string)

    return final_text_list

#2 Train a KNN classifier

In [6]:
from sklearn.model_selection import train_test_split

X=train_df[["text"]]
Y=train_df["label"]
X_train, X_val, y_train, y_val = train_test_split(X,
                                                  Y,
                                                  test_size=0.10,
                                                  shuffle=True,
                                                  random_state=324
                                                 )

In [7]:
print("Processing the text fields")
train_text_list = process_text(X_train["text"].tolist())
val_text_list = process_text(X_val["text"].tolist())

Processing the text fields


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier


import gensim
from gensim.models import Word2Vec
### PIPELINE ###
##########################
w2v = gensim.models.Word2Vec()
pipeline = Pipeline([
    ('text_vect', CountVectorizer(binary=True,
    #('text_vect', TfidfVectorizer(use_idf=True,
                                  max_features=10)),
    ('knn', KNeighborsClassifier())
                                ])


# Visualize the pipeline
# This will come in handy especially when building more complex pipelines, stringing together multiple preprocessing steps
from sklearn import set_config
set_config(display='diagram')
pipeline

Pipeline(steps=[('text_vect', CountVectorizer(binary=True, max_features=10)),
                ('knn', KNeighborsClassifier())])

In [ ]:
# We using lists of processed text fields
X_train = train_text_list
X_val = val_text_list

# Fit the Pipeline to training data
pipeline.fit(X_train, y_train.values)

Pipeline(steps=[('text_vect', CountVectorizer(binary=True, max_features=10)),
                ('knn', KNeighborsClassifier())])

#3. Make predictions on your test dataset

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

# Use the fitted pipeline to make predictions on the validation dataset
val_predictions = pipeline.predict(X_val)
print(confusion_matrix(y_val.values, val_predictions))
print(classification_report(y_val.values, val_predictions))
print("Accuracy (validation):", accuracy_score(y_val.values, val_predictions))

[[669 611]
 [631 589]]
              precision    recall  f1-score   support

           0       0.51      0.52      0.52      1280
           1       0.49      0.48      0.49      1220

    accuracy                           0.50      2500
   macro avg       0.50      0.50      0.50      2500
weighted avg       0.50      0.50      0.50      2500

Accuracy (validation): 0.5032


#4. Experimentations

#1) Task 1

## CountVectorizer(binary=False)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier


import gensim
from gensim.models import Word2Vec
### PIPELINE ###
##########################
w2v = gensim.models.Word2Vec()
pipeline = Pipeline([
    ('text_vect', CountVectorizer(binary=False,
    #( 'text_vect', TfidfVectorizer(use_idf=True,
                                  max_features=10)),
    ('knn', KNeighborsClassifier())
                                ])


# Visualize the pipeline
# This will come in handy especially when building more complex pipelines, stringing together multiple preprocessing steps
from sklearn import set_config
set_config(display='diagram')
pipeline

# We using lists of processed text fields
X_train = train_text_list
X_val = val_text_list

# Fit the Pipeline to training data
pipeline.fit(X_train, y_train.values)

from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

# Use the fitted pipeline to make predictions on the validation dataset
val_predictions = pipeline.predict(X_val)
print(confusion_matrix(y_val.values, val_predictions))
print(classification_report(y_val.values, val_predictions))
print("Accuracy (validation):", accuracy_score(y_val.values, val_predictions))

[[670 610]
 [589 631]]
              precision    recall  f1-score   support

           0       0.53      0.52      0.53      1280
           1       0.51      0.52      0.51      1220

    accuracy                           0.52      2500
   macro avg       0.52      0.52      0.52      2500
weighted avg       0.52      0.52      0.52      2500

Accuracy (validation): 0.5204


Observation: When binary = false, the f1-score and the accuracy increase.

## TfidfVectorizer(use_idf=True)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier


import gensim
from gensim.models import Word2Vec
### PIPELINE ###
##########################
w2v = gensim.models.Word2Vec()
pipeline = Pipeline([
    #('text_vect', CountVectorizer(binary=False,
    ( 'text_vect', TfidfVectorizer(use_idf=True,
                                  max_features=10)),
    ('knn', KNeighborsClassifier())
                                ])


# Visualize the pipeline
# This will come in handy especially when building more complex pipelines, stringing together multiple preprocessing steps
from sklearn import set_config
set_config(display='diagram')
pipeline

# We using lists of processed text fields
X_train = train_text_list
X_val = val_text_list

# Fit the Pipeline to training data
pipeline.fit(X_train, y_train.values)

from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

# Use the fitted pipeline to make predictions on the validation dataset
val_predictions = pipeline.predict(X_val)
print(confusion_matrix(y_val.values, val_predictions))
print(classification_report(y_val.values, val_predictions))
print("Accuracy (validation):", accuracy_score(y_val.values, val_predictions))

[[651 629]
 [607 613]]
              precision    recall  f1-score   support

           0       0.52      0.51      0.51      1280
           1       0.49      0.50      0.50      1220

    accuracy                           0.51      2500
   macro avg       0.51      0.51      0.51      2500
weighted avg       0.51      0.51      0.51      2500

Accuracy (validation): 0.5056


Observation: when we switch the tifdfvectorizer, the value of the f1-score and the accuracy decrease.

#2) Task 2

## CountVectorizer(binary=True, max_features=100)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier


import gensim
from gensim.models import Word2Vec
### PIPELINE ###
##########################
w2v = gensim.models.Word2Vec()
pipeline = Pipeline([
    ('text_vect', CountVectorizer(binary=True,
    #( 'text_vect', TfidfVectorizer(use_idf=True,
                                  max_features=100)),
    ('knn', KNeighborsClassifier())
                                ])


# Visualize the pipeline
# This will come in handy especially when building more complex pipelines, stringing together multiple preprocessing steps
from sklearn import set_config
set_config(display='diagram')
pipeline

# We using lists of processed text fields
X_train = train_text_list
X_val = val_text_list

# Fit the Pipeline to training data
pipeline.fit(X_train, y_train.values)

from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

# Use the fitted pipeline to make predictions on the validation dataset
val_predictions = pipeline.predict(X_val)
print(confusion_matrix(y_val.values, val_predictions))
print(classification_report(y_val.values, val_predictions))
print("Accuracy (validation):", accuracy_score(y_val.values, val_predictions))

[[795 485]
 [436 784]]
              precision    recall  f1-score   support

           0       0.65      0.62      0.63      1280
           1       0.62      0.64      0.63      1220

    accuracy                           0.63      2500
   macro avg       0.63      0.63      0.63      2500
weighted avg       0.63      0.63      0.63      2500

Accuracy (validation): 0.6316


## CountVectorizer(binary=True, max_features=1000)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier


import gensim
from gensim.models import Word2Vec
### PIPELINE ###
##########################
w2v = gensim.models.Word2Vec()
pipeline = Pipeline([
    ('text_vect', CountVectorizer(binary=True,
    #( 'text_vect', TfidfVectorizer(use_idf=True,
                                  max_features=1000)),
    ('knn', KNeighborsClassifier())
                                ])


# Visualize the pipeline
# This will come in handy especially when building more complex pipelines, stringing together multiple preprocessing steps
from sklearn import set_config
set_config(display='diagram')
pipeline

# We using lists of processed text fields
X_train = train_text_list
X_val = val_text_list

# Fit the Pipeline to training data
pipeline.fit(X_train, y_train.values)

from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

# Use the fitted pipeline to make predictions on the validation dataset
val_predictions = pipeline.predict(X_val)
print(confusion_matrix(y_val.values, val_predictions))
print(classification_report(y_val.values, val_predictions))
print("Accuracy (validation):", accuracy_score(y_val.values, val_predictions))

[[757 523]
 [401 819]]
              precision    recall  f1-score   support

           0       0.65      0.59      0.62      1280
           1       0.61      0.67      0.64      1220

    accuracy                           0.63      2500
   macro avg       0.63      0.63      0.63      2500
weighted avg       0.63      0.63      0.63      2500

Accuracy (validation): 0.6304


## TfidfVectorizer(use_idf=True, max_features=100)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier


import gensim
from gensim.models import Word2Vec
### PIPELINE ###
##########################
w2v = gensim.models.Word2Vec()
pipeline = Pipeline([
    #('text_vect', CountVectorizer(binary=True,
    ( 'text_vect', TfidfVectorizer(use_idf=True,
                                  max_features=100)),
    ('knn', KNeighborsClassifier())
                                ])


# Visualize the pipeline
# This will come in handy especially when building more complex pipelines, stringing together multiple preprocessing steps
from sklearn import set_config
set_config(display='diagram')
pipeline

# We using lists of processed text fields
X_train = train_text_list
X_val = val_text_list

# Fit the Pipeline to training data
pipeline.fit(X_train, y_train.values)

from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

# Use the fitted pipeline to make predictions on the validation dataset
val_predictions = pipeline.predict(X_val)
print(confusion_matrix(y_val.values, val_predictions))
print(classification_report(y_val.values, val_predictions))
print("Accuracy (validation):", accuracy_score(y_val.values, val_predictions))

[[818 462]
 [381 839]]
              precision    recall  f1-score   support

           0       0.68      0.64      0.66      1280
           1       0.64      0.69      0.67      1220

    accuracy                           0.66      2500
   macro avg       0.66      0.66      0.66      2500
weighted avg       0.66      0.66      0.66      2500

Accuracy (validation): 0.6628


## TfidfVectorizer(use_idf=True, max_features=1000)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier


import gensim
from gensim.models import Word2Vec
### PIPELINE ###
##########################
w2v = gensim.models.Word2Vec()
pipeline = Pipeline([
    #('text_vect', CountVectorizer(binary=True,
    ( 'text_vect', TfidfVectorizer(use_idf=True,
                                  max_features=1000)),
    ('knn', KNeighborsClassifier())
                                ])


# Visualize the pipeline
# This will come in handy especially when building more complex pipelines, stringing together multiple preprocessing steps
from sklearn import set_config
set_config(display='diagram')
pipeline

# We using lists of processed text fields
X_train = train_text_list
X_val = val_text_list

# Fit the Pipeline to training data
pipeline.fit(X_train, y_train.values)

from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

# Use the fitted pipeline to make predictions on the validation dataset
val_predictions = pipeline.predict(X_val)
print(confusion_matrix(y_val.values, val_predictions))
print(classification_report(y_val.values, val_predictions))
print("Accuracy (validation):", accuracy_score(y_val.values, val_predictions))

[[863 417]
 [291 929]]
              precision    recall  f1-score   support

           0       0.75      0.67      0.71      1280
           1       0.69      0.76      0.72      1220

    accuracy                           0.72      2500
   macro avg       0.72      0.72      0.72      2500
weighted avg       0.72      0.72      0.72      2500

Accuracy (validation): 0.7168


Observation: when i use the tfidfvectorizer and max_features = 1000, i obtain the biggest value of the experiment for the f1-score and the accuracy. When the countervectorizer is binary = true with max_features = 100, the f1-score take the same value for each review.

#3) Task 3

## TfidfVectorizer(use_idf=True, max_features=1000, n_neighbors=10)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier


import gensim
from gensim.models import Word2Vec
### PIPELINE ###
##########################
w2v = gensim.models.Word2Vec()
pipeline = Pipeline([
    #('text_vect', CountVectorizer(binary=True,
    ( 'text_vect', TfidfVectorizer(use_idf=True,
                                  max_features=1000)),
    ('knn', KNeighborsClassifier(n_neighbors=10))
                                ])


# Visualize the pipeline
# This will come in handy especially when building more complex pipelines, stringing together multiple preprocessing steps
from sklearn import set_config
set_config(display='diagram')
pipeline

# We using lists of processed text fields
X_train = train_text_list
X_val = val_text_list

# Fit the Pipeline to training data
pipeline.fit(X_train, y_train.values)

from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

# Use the fitted pipeline to make predictions on the validation dataset
val_predictions = pipeline.predict(X_val)
print(confusion_matrix(y_val.values, val_predictions))
print(classification_report(y_val.values, val_predictions))
print("Accuracy (validation):", accuracy_score(y_val.values, val_predictions))

[[962 318]
 [342 878]]
              precision    recall  f1-score   support

           0       0.74      0.75      0.74      1280
           1       0.73      0.72      0.73      1220

    accuracy                           0.74      2500
   macro avg       0.74      0.74      0.74      2500
weighted avg       0.74      0.74      0.74      2500

Accuracy (validation): 0.736


## TfidfVectorizer(use_idf=True, max_features=1000, n_neighbors=100)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier


import gensim
from gensim.models import Word2Vec
### PIPELINE ###
##########################
w2v = gensim.models.Word2Vec()
pipeline = Pipeline([
    #('text_vect', CountVectorizer(binary=True,
    ( 'text_vect', TfidfVectorizer(use_idf=True,
                                  max_features=1000)),
    ('knn', KNeighborsClassifier(n_neighbors=100))
                                ])


# Visualize the pipeline
# This will come in handy especially when building more complex pipelines, stringing together multiple preprocessing steps
from sklearn import set_config
set_config(display='diagram')
pipeline

# We using lists of processed text fields
X_train = train_text_list
X_val = val_text_list

# Fit the Pipeline to training data
pipeline.fit(X_train, y_train.values)

from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

# Use the fitted pipeline to make predictions on the validation dataset
val_predictions = pipeline.predict(X_val)
print(confusion_matrix(y_val.values, val_predictions))
print(classification_report(y_val.values, val_predictions))
print("Accuracy (validation):", accuracy_score(y_val.values, val_predictions))

[[ 974  306]
 [ 213 1007]]
              precision    recall  f1-score   support

           0       0.82      0.76      0.79      1280
           1       0.77      0.83      0.80      1220

    accuracy                           0.79      2500
   macro avg       0.79      0.79      0.79      2500
weighted avg       0.79      0.79      0.79      2500

Accuracy (validation): 0.7924


## TfidfVectorizer(use_idf=True, max_features=1000, n_neighbors=1000)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier


import gensim
from gensim.models import Word2Vec
### PIPELINE ###
##########################
w2v = gensim.models.Word2Vec()
pipeline = Pipeline([
    #('text_vect', CountVectorizer(binary=True,
    ( 'text_vect', TfidfVectorizer(use_idf=True,
                                  max_features=1000)),
    ('knn', KNeighborsClassifier(n_neighbors=1000))
                                ])


# Visualize the pipeline
# This will come in handy especially when building more complex pipelines, stringing together multiple preprocessing steps
from sklearn import set_config
set_config(display='diagram')
pipeline

# We using lists of processed text fields
X_train = train_text_list
X_val = val_text_list

# Fit the Pipeline to training data
pipeline.fit(X_train, y_train.values)

from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

# Use the fitted pipeline to make predictions on the validation dataset
val_predictions = pipeline.predict(X_val)
print(confusion_matrix(y_val.values, val_predictions))
print(classification_report(y_val.values, val_predictions))
print("Accuracy (validation):", accuracy_score(y_val.values, val_predictions))

[[1021  259]
 [ 243  977]]
              precision    recall  f1-score   support

           0       0.81      0.80      0.80      1280
           1       0.79      0.80      0.80      1220

    accuracy                           0.80      2500
   macro avg       0.80      0.80      0.80      2500
weighted avg       0.80      0.80      0.80      2500

Accuracy (validation): 0.7992


## TfidfVectorizer(use_idf=True, max_features=1000, n_neighbors=10000)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier


import gensim
from gensim.models import Word2Vec
### PIPELINE ###
##########################
w2v = gensim.models.Word2Vec()
pipeline = Pipeline([
    #('text_vect', CountVectorizer(binary=True,
    ( 'text_vect', TfidfVectorizer(use_idf=True,
                                  max_features=1000)),
    ('knn', KNeighborsClassifier(n_neighbors=10000))
                                ])


# Visualize the pipeline
# This will come in handy especially when building more complex pipelines, stringing together multiple preprocessing steps
from sklearn import set_config
set_config(display='diagram')
pipeline

# We using lists of processed text fields
X_train = train_text_list
X_val = val_text_list

# Fit the Pipeline to training data
pipeline.fit(X_train, y_train.values)

from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

# Use the fitted pipeline to make predictions on the validation dataset
val_predictions = pipeline.predict(X_val)
print(confusion_matrix(y_val.values, val_predictions))
print(classification_report(y_val.values, val_predictions))
print("Accuracy (validation):", accuracy_score(y_val.values, val_predictions))

[[1123  157]
 [ 390  830]]
              precision    recall  f1-score   support

           0       0.74      0.88      0.80      1280
           1       0.84      0.68      0.75      1220

    accuracy                           0.78      2500
   macro avg       0.79      0.78      0.78      2500
weighted avg       0.79      0.78      0.78      2500

Accuracy (validation): 0.7812


Observation: as the value of n_neighbors increases, the accuracy increases, except when I tested the value n_neighbors = 10000. For the n_neighbors = 1000, we see the same value for the f1-score and the accuracy. That is the optimized value for the n_neighbor. I observe a difference between the f1-score and accuracy of n_neighbor = 10 and that of n_neighbor = 100. However, the difference reduces between n_neighbor = 100 and n_neighbor = 1000.